# Logistic regresion Review

 - binary classification (Classification algorithm)
 - for example : Spam/ email , Customers (laon default),...
 - predict two Classes 0 and 1 
 - we use "Sigmoid" function or logistic functions takes in any value and output it to be between 0 and 1 
 - The difference between Linear Regression and logistic Regression is that we can say we take our linear Regression Solution and we put it into sigmoid Funtion.
 - we can set a cutoff point at 0.5 anything below the result be 0 and anything above the result will be 1 
 - Evalution Logistic Regression (Matric Confusion)
 
 - accurary = somme (True positive +True Negative) / Total
 - Recall = True positive / (True Positive + Faux negative)
 - ... the confusion can lead us to all this metrics
 - the good metrics will really depends on the specific situation sometime we take the accuracy some time we 
 take the recall

###  accuracy 
answer the question how often is it correct  if have a balanced dataset it s good 
### Error Rate (Miss classification Rate)
how often  we are wrong 

# Curve ROC 
is just a plot sensitivite (True Postive Rate) versus  - specificity (Fakse Positive Rate )

# Code an Example 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("logReg_test").getOrCreate()

In [16]:
from pyspark.ml.classification import LogisticRegression

In [19]:
my_data = spark.read.format("libsvm").load("../../Data/sample_libsvm_data.txt")

In [20]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



# create our logistic Regression Model

In [21]:
lgr = LogisticRegression()

In [22]:
lgr_model = lgr.fit(my_data)

In [23]:
log_summary = lgr_model.summary

In [25]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [26]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111051...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170301...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200602...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695578...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102648...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

# Using Evaluation 

In [27]:
train_data, test_data = my_data.randomSplit([0.7,.3])

In [28]:
lgr = LogisticRegression()
lgr_model = lgr.fit(train_data)
log_summary = lgr_model.summary
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [35]:
prediction = lgr_model.evaluate(test_data)
prediction.predictions.show()


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[10.6043582154818...|[0.99997519295872...|       0.0|
|  0.0|(692,[121,122,123...|[23.7466076879632...|[0.99999999995136...|       0.0|
|  0.0|(692,[123,124,125...|[33.0453709442870...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[21.5867548664121...|[0.99999999957831...|       0.0|
|  0.0|(692,[126,127,128...|[25.8996372486567...|[0.99999999999435...|       0.0|
|  0.0|(692,[126,127,128...|[32.8022375408048...|[0.99999999999999...|       0.0|
|  0.0|(692,[127,128,129...|[22.9373802966879...|[0.99999999989074...|       0.0|
|  0.0|(692,[150,151,152...|[17.7584051400711...|[0.99999998060799...|       0.0|
|  0.0|(692,[151,152,153...|[28.8133134682893...|[0.99999999999969...|       0.0|
|  0.0|(692,[152

In [39]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                MulticlassClassificationEvaluator)

In [40]:
my_eval = BinaryClassificationEvaluator()

In [44]:
my_final_roc = my_eval.evaluate(prediction.predictions)
my_final_roc

1.0

 this mean that the erea on the curve Roc is 1 what is a good sigh that we succed to predict all truly the binary class 

# Realistic Exercice Titanic 

Attempt to predict what passenger survived the titanic crush based solely on passenger's
features (age, cabin, children,...)
- deal with missing value , .... 

In [46]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("logReg_test").getOrCreate()
from pyspark.ml.classification import LogisticRegression

In [49]:
my_data = spark.read.csv("../../Data/titanic.csv",inferSchema=True, header= True)
my_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Select only the features we need 
we can use after that feature angering for age for example, transform categorical feaeture to numeric or binary

In [52]:
my_data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [54]:
my_cols = my_data.select(["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"])

## Dealing with the missing data 
to go so fast just here we will delete the missing data but there a lot of solution for that (imputation of missing value ) and drop them directly are the most easiest and worst thing to do 

In [55]:
my_final_data = my_cols.na.drop()

In [79]:
## Prepare our data to fit the Logistic Regression Algorithm

# one hot Encoder to transform categorical feature to binary  feature
# StringIndexer to to transform categorical feature to numeric 
from pyspark.ml.feature import (VectorAssembler , VectorIndexer, OneHotEncoder, StringIndexer)

In [80]:
gender_indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
gender_encoder = OneHotEncoder(inputCol="SexIndex", outputCol="SexVec")

embark_indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
embark_encoder = OneHotEncoder(inputCol="EmbarkedIndex", outputCol="EmbarkedVec")

In [82]:
assember = VectorAssembler(inputCols=["Pclass","SexVec","EmbarkedVec",
                                    "Age","SibSp","Parch","Fare"], outputCol="features")

# Using Pipeline 

In [83]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [84]:
log_reg_titanic = LogisticRegression(featuresCol="features", labelCol="Survived")

In [85]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer,gender_encoder,embark_encoder,
                             assember,log_reg_titanic])

In [86]:
train_data, test_data = my_final_data.randomSplit([0.7,.3])

In [87]:
fit_model = pipeline.fit(train_data)

In [88]:
result = fit_model.transform(test_data)

# Evaluation

In [89]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [93]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Survived")

In [97]:
result.select("Survived","prediction").show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [99]:
AUC = my_eval.evaluate(result)
AUC

0.78586464992983